In [1]:
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-3.20.1-cp39-cp39-win_amd64.whl (904 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.9
    Uninstalling protobuf-4.21.9:
      Successfully uninstalled protobuf-4.21.9


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.50.0 requires protobuf>=4.21.6, but you have protobuf 3.20.1 which is incompatible.
googleapis-common-protos 1.57.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-cloud-bigquery 3.4.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-cloud-bigquery-storage 2.16.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.1 which is incompatible.
google-api-core 2.10.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 

In [2]:
import mysql.connector

In [3]:
cnx=mysql.connector.connect(user='root',password='123456',host='34.75.22.238',database='sukyanya')
cursor=cnx.cursor()

In [4]:
customer_master='''CREATE TABLE CUSTOMER_MASTER (
Customerid int(100) PRIMARY KEY AUTO_INCREMENT,
Name varchar(255) NOT NULL,
Address varchar(500) NOT NULL,
City varchar(20) NOT NULL,
State varchar(20) NOT NULL,
Pincode int(6) NOT NULL,
Update_timestamp datetime NOT NULL
);'''
cursor.execute(customer_master)

ProgrammingError: 1050 (42S01): Table 'CUSTOMER_MASTER' already exists

In [61]:
product_master='''create table PRODUCT_MASTER (
Productid int(100) primary key auto_increment,
Productcode varchar(100) not null,
Productname varchar(255) not null,
Sku varchar(100) not null,
Rate int(100) not null,
Isactive varchar(5) not null
);'''
cursor.execute(product_master)

In [5]:
try:
    order_details='''CREATE TABLE ORDER_DETAILS(
    Orderid int(100) NOT NULL,
    Customerid int(100) NOT NULL,
    Order_status_update_timestamp datetime NOT NULL,
    Order_status varchar(100) NOT NULL,
    CONSTRAINT ORDER_PK PRIMARY KEY (Orderid, Order_status) ,
    FOREIGN KEY (Customerid) REFERENCES CUSTOMER_MASTER(Customerid)
    );'''
    cursor.execute(order_details)

except Exception as e:
    print(e)

1050 (42S01): Table 'ORDER_DETAILS' already exists


In [6]:
try:
    order_items='''CREATE TABLE ORDER_ITEMS(
    Orderid int(100) NOT NULL,
    Productid int(100) NOT NULL,
    Quantity int(255) NOT NULL,
    CONSTRAINT ORDER_ITEMS_PK PRIMARY KEY(Orderid, Productid),
    FOREIGN KEY (Orderid) REFERENCES ORDER_DETAILS(Orderid),
    FOREIGN KEY (Productid) REFERENCES PRODUCT_MASTER(Productid)
    );'''
    cursor.execute(order_items)

except Exception as e:
    print(e)

1050 (42S01): Table 'ORDER_ITEMS' already exists


In [24]:
cursor.close()
cnx.close()

In [87]:
pip install faker

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [8]:
from faker import Faker
fake = Faker('en_IN')

# Customer table generation

In [9]:
cust_fields= ['customerid','name','address','city','state','pincode','update_timestamp']

In [10]:
states = {'Madhya Pradesh':['Indore','Bhopal','Gwalior','Ujjain','Ratlam'],
          'Maharashtra':['Mumbai','Nagpur','Pune'],
          'Gujarat' : ['Vadodara','Ahmedabad','Surat','Gandhi Nagar'],
          'Karnataka':['Bangalore','Mangalore','Mysore'],
          'Uttar Pradesh':['Noida','Lucknow','Mathura'],
          'Jharkhand' : ['Ranchi'],
          'Chhattisgarh':['Raipur']}

In [11]:
customer = pd.DataFrame(columns=cust_fields,index=range(1,1000))
Faker.seed(10)
for i in range(1,1000):
    customer['customerid'][i]=i
    customer['name'][i]=fake.name()
    customer['address'][i]=fake.street_address()    
    customer['state'][i]=fake.random_element(states.keys())
    customer['city'][i]=fake.random_element(states[customer['state'][i]])
    customer['pincode'][i]=fake.postcode()
    customer['update_timestamp'][i]=fake.date_time_this_year()

In [12]:
from sqlalchemy import create_engine

In [13]:
import sqlalchemy, pymysql
from sqlalchemy.dialects import mysql
import pandas as pd
db = sqlalchemy.create_engine(
     sqlalchemy.engine.url.URL(
          drivername='mysql+pymysql',
          username='root',
          password="123456",
          host='34.75.22.238',
          database="sukyanya"
     )
)

C:\Users\sukanya.sahay\AppData\Local\Temp\ipykernel_21772\415947101.py:5: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  sqlalchemy.engine.url.URL(


# Loading data

In [59]:
customer.to_sql('CUSTOMER_MASTER', con=db, if_exists='append',index=False)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1' for key 'CUSTOMER_MASTER.PRIMARY'")
[SQL: INSERT INTO `CUSTOMER_MASTER` (customerid, name, address, city, state, pincode, update_timestamp) VALUES (%(customerid)s, %(name)s, %(address)s, %(city)s, %(state)s, %(pincode)s, %(update_timestamp)s)]
[parameters: ({'customerid': 1, 'name': 'Aaryahi Khurana', 'address': '774, Badal Street', 'city': 'Lucknow', 'state': 'Uttar Pradesh', 'pincode': '135062', 'update_timestamp': datetime.datetime(2022, 8, 23, 6, 8, 38)}, {'customerid': 2, 'name': 'Vaibhav Kade', 'address': '47/245\nKari Street', 'city': 'Raipur', 'state': 'Chhattisgarh', 'pincode': '960903', 'update_timestamp': datetime.datetime(2022, 2, 22, 0, 42, 52)}, {'customerid': 3, 'name': 'Romil Loyal', 'address': '587\nKeer Marg', 'city': 'Ratlam', 'state': 'Madhya Pradesh', 'pincode': '823630', 'update_timestamp': datetime.datetime(2022, 1, 13, 7, 54, 2)}, {'customerid': 4, 'name': 'Romil Taneja', 'address': '12/69,

Loading customer to customer_master

In [14]:
qry='SELECT * from CUSTOMER_MASTER;'

In [15]:
result= pd.read_sql(qry,con=db)

In [16]:
result

,Customerid,Name,Address,City,State,Pincode,Update_timestamp
0,1,Aaryahi Khurana,"774, Badal Street",Lucknow,Uttar Pradesh,135062,2022-08-23 06:08:38
1,2,Vaibhav Kade,47/245\nKari Street,Mysore,Karnataka,90323,2022-04-28 14:51:54
2,3,Trisha Yadav,"H.No. 87, Keer Marg",Vadodara,Gujrat,823630,2022-01-13 07:54:02
3,4,Romil Taneja,"12/69, Rama Path",Belgaum,Karnataka,723566,2022-06-23 01:21:38
4,5,Pihu Devan,"82/14, Dora Chowk",Bengaluru,Karnataka,755630,2022-07-31 14:22:30
...,...,...,...,...,...,...,...
994,995,Anvi Iyer,"H.No. 639, Khare Ganj",Kolkata,West Bengal,400698,2022-04-17 04:31:55
995,996,Amira Ramaswamy,"43/034, Tara Path",Mysore,Karnataka,747381,2022-07-06 02:52:08
996,997,Miraan Lanka,"H.No. 565, Krishna Marg",Chennai,Tamil Nadu,87280,2022-07-11 10:00:09
997,998,Fateh Mangal,"983, Sheth Chowk",Kolkata,West Bengal,936031,2022-05-11 06:05:25


# Product Master Table

In [17]:
product_attributes = ['Productid','Productcode','Productname','Sku','Rate','Isactive']

In [18]:
products = ['Organic India Chyavanprash',
            'Aashirwad Atta',
            'Herbal Strategi Oil',
            'Herbal Strategi Floor Cleaner',
            'Soulfull Ragi Bites Choco Fills',
            'Natureland Puffed Wheat',
            'Natureland Pasta Macroni',
            'Natureland Rice Poha',
            'Pure Sure Organic Chakli',
            'Organic Wellness Chilli Chocolate',
            'Natureland Green Tea',
            'Organic India Tulsi Tummy',
            'Organic Wellness Turmeric Chocolate',
            'Organic Wellness Ginger Tea',
            '24 Mantra Organic Orange Juice',
            '24 Mantra Organic Mango Juice',
            'Ayuda Organics Desi Jaggery',
            '24 Mantra Handpounded Rice',
            '24 Mantra Idly Rice',
            'Natureland Brown Sugar',
            'Natureland Sunflower Oil',
            'Natureland Mustard Oil',
            'Natureland Jaggery Powder',
            'Natureland Red Rice',
            'Natureland Rice Flour',
            'Biotique Bio White Face Wash',
            'Vitro Naturals Aloe Face Wipes',
            'Biotique Bio Honey Gel Face Wash',
            'Biotique Bio Orris Root',
            'Vitro Naturals Aloe Face Wash',
            'BioPure Hand Sanitizer',
            'Soultree Lipgloss Nude Pink',
            'Soultree Lipgloss Rich Earth',
            'Soultree Lipgloss Lush Berry',
            'Soultree Lipstick Iced Plum',
            'Soultree Kajal Pure Black',
            'Soultree Grey Glow Kajal',
            'Ayuda Organics Chhundo (Sun Dried)',
            'Ayuda Organics Raw Honey',
            'Ayuda Organics Homemade Mango Pickle',
            'Ayuda Organics Homemade Chana Methi Pickle',
            'Natureland Garlic Pickle',
            'Natureland Pineapple Jam',
            'Natureland Pineapple Jam',
            'Natureland Honey',
            'Natureland Walnut',
            'Natureland Cashew',
            'Natureland Raisins',
            'Natureland Almonds',
            'Pure Sure Organic Rice Dosa Mix'
           ]

In [19]:
product = pd.DataFrame(columns = product_attributes, index=range(1,101))
Faker.seed(10)
for i in range(1,26):
    product_name = products[i-1]
    for j in range((i*4)-3,(i*4)+1):        
        product['Productid'][j]=j
        product['Productcode'][j]=fake.bothify('????-########', letters='ABCDE')
        product['Productname'][j]= product_name
        sku = 0
        if j%4 == 1:
            sku = fake.random_int(min=1, max=10)
        else:
            sku = int(product['Sku'][j-1][:-2]) + 2
        product['Sku'][j] = str(sku) + 'KG'
        product['Rate'][j]=round(np.random.normal(1000,200))
        product['Isactive'][j]=fake.boolean()

Loading Product_master

In [20]:
product.to_sql('PRODUCT_MASTER', con=db, if_exists='append',index=False)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1' for key 'PRODUCT_MASTER.PRIMARY'")
[SQL: INSERT INTO `PRODUCT_MASTER` (`Productid`, `Productcode`, `Productname`, `Sku`, `Rate`, `Isactive`) VALUES (%(Productid)s, %(Productcode)s, %(Productname)s, %(Sku)s, %(Rate)s, %(Isactive)s)]
[parameters: ({'Productid': 1, 'Productcode': 'DCBA-90679037', 'Productname': 'Organic India Chyavanprash', 'Sku': '9KG', 'Rate': 1290, 'Isactive': 0}, {'Productid': 2, 'Productcode': 'CDDC-51350629', 'Productname': 'Organic India Chyavanprash', 'Sku': '11KG', 'Rate': 1176, 'Isactive': 0}, {'Productid': 3, 'Productcode': 'DEDA-47245273', 'Productname': 'Organic India Chyavanprash', 'Sku': '13KG', 'Rate': 1094, 'Isactive': 0}, {'Productid': 4, 'Productcode': 'CEDD-03234853', 'Productname': 'Organic India Chyavanprash', 'Sku': '15KG', 'Rate': 1268, 'Isactive': 0}, {'Productid': 5, 'Productcode': 'AADC-19582363', 'Productname': 'Aashirwad Atta', 'Sku': '10KG', 'Rate': 586, 'Isactive': 0}, 

In [21]:
qry='select * from PRODUCT_MASTER'

In [22]:
result= pd.read_sql(qry,con=db)

In [23]:
result.head(20)

,Productid,Productcode,Productname,Sku,Rate,Isactive
0,1,DCBA-90679037,Organic India Chyavanprash,9KG,1082,0
1,2,CDDC-51350629,Organic India Chyavanprash,11KG,845,0
2,3,DEDA-47245273,Organic India Chyavanprash,13KG,812,0
3,4,CEDD-03234853,Organic India Chyavanprash,15KG,692,0
4,5,AADC-19582363,Aashirwad Atta,10KG,1058,0
5,6,BAAD-18126959,Aashirwad Atta,12KG,1174,1
6,7,EEBA-35667342,Aashirwad Atta,14KG,760,1
7,8,DBAE-74222755,Aashirwad Atta,16KG,1331,0
8,9,EBDD-05531476,Herbal Strategi Oil,4KG,1037,0
9,10,DDDB-84879712,Herbal Strategi Oil,6KG,980,0


# Order Details Table

In [24]:
order_details_attribute = ['Orderid', 'Customerid', 'Order_status_update_timestamp', 'Order_status']

Loading order table

In [25]:
order_detail_fields = ['orderid', 'customerid', 'order_status_update_timestamp', 'order_status']
orders_Received = pd.DataFrame(columns=order_detail_fields, index=range(1, 60001))

x=1
j=1
for i in range(1, 20001):
    f=fake.random_element(customer['customerid'])
    for k in range (j,j+3):
        orders_Received['orderid'][k] = x
        orders_Received['customerid'][k]=f

    orders_Received['order_status_update_timestamp'][j] = fake.date_time_this_year()
    orders_Received['order_status'][j] = 'Received'
    orders_Received['order_status_update_timestamp'][j+1] = orders_Received['order_status_update_timestamp'][j] + timedelta(seconds=fake.random_int(min=1, max=86400))
    orders_Received['order_status'][j+1] = 'In_progress'
    orders_Received['order_status_update_timestamp'][j+2] = orders_Received['order_status_update_timestamp'][j+1] + timedelta(seconds=fake.random_int(min=1, max=172800))
    orders_Received['order_status'][j+2] = 'Delivered'
    j+=3
    x+=1

In [26]:
qry='select * from ORDER_DETAILS'

In [27]:
result= pd.read_sql(qry,con=db)

In [28]:
result.head()

,Orderid,Customerid,Order_status_update_timestamp,Order_status
0,1,854,2022-04-01 03:13:31,Delivered
1,1,854,2022-03-30 12:05:21,In_progress
2,1,854,2022-03-30 00:32:55,Received
3,2,403,2022-01-18 11:44:52,Delivered
4,2,403,2022-01-17 00:26:58,In_progress


# Order Items Table

In [29]:
order_item_fields = ['orderid','productid','quantity']

In [30]:
order_items = pd.DataFrame(columns=order_item_fields, index = range(1,50000))

In [31]:
for i in range(1,50000):
    order_items['orderid'][i]=fake.random_element(orders_Received['orderid'])
    order_items['productid'][i]=fake.random_element(product['Productid'])
    order_items['quantity'][i]=fake.random_int(min=1, max=20)

In [32]:
order_items['orderid'][i]

7176

In [33]:
from google.oauth2 import service_account

In [111]:
!pip install pandas-gbq -U

Defaulting to user installation because normal site-packages is not writeable


In [34]:
credentials = service_account.Credentials.from_service_account_file(r"C:\Users\sukanya.sahay\Downloads\pax-8-366608-91c75e747913.json")

from faker import Faker
fake= Faker('en_IN')
import numpy as np
import pandas as pd
import pandas_gbq

cust_fields= ['customerid','name','address','city','state','pincode','update_timestamp']
states = {'Gujrat':['Ahmadabad','Surat','Vadodara'],
          'Karnataka':['Belgaum','Bengaluru','Mangalore','Mysore'],
          'Maharashtra':['Mumbai','Nagpur','Navi Mumbai','Pune'],
          'Tamil Nadu':['Chennai','Coimbatore','Madurai'],
          'Uttar Pradesh':['Agra','Greater Noida','Kanpur','Lucknow','Mathura'],
          'West Bengal':['Kolkata']}
customer = pd.DataFrame(columns=cust_fields,index=range(1,1000))
Faker.seed(10)
for i in range(1,1000):
    customer['customerid'][i]=i
    customer['name'][i]=fake.name()
    customer['address'][i]=fake.street_address()
    customer['state'][i]=fake.random_element(states.keys())
    customer['city'][i]=fake.random_element(states[customer['state'][i]])
    customer['pincode'][i]=fake.postcode()
    customer['update_timestamp'][i]=fake.date_time_this_year()
print('a')


a


In [35]:
cols=['customerid',"pincode",'update_timestamp']
for i in cols:
    customer[i]= customer[i].map(str)
print(customer.dtypes)
project_id='pax-8-366608'
table='test.test'
pandas_gbq.to_gbq(customer, table, project_id=project_id,if_exists='replace',credentials=credentials)

customerid          object
name                object
address             object
city                object
state               object
pincode             object
update_timestamp    object
dtype: object


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1001.74it/s]


In [36]:
from google.cloud import bigquery

In [115]:
pip install git+https://github.com/pydata/pandas-gbq.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/pydata/pandas-gbq.git to c:\users\sukanya.sahay\appdata\local\temp\pip-req-build-97_9xlax
Note: you may need to restart the kernel to use updated packages.


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [37]:
import pandas_gbq

In [38]:
import pydata_google_auth

In [119]:
pip install pydata-google-auth

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
